## Neural Network w/o NN library

In [1]:
import math
import matplotlib.pyplot as plt 
import numpy as np 
import torchvision
from torchvision import transforms, datasets
# Loading code from https://www.youtube.com/watch?v=ixathu7U-LQ&t=1040s
# 2 lines
train = datasets.MNIST('', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))


test = datasets.MNIST('', train=False, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

# Splitting into data and labels
# 
trainset = train.data.numpy()
y_train = train.targets.numpy()

testset = test.data.numpy()
y_test = test.targets.numpy()

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw


Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...


/Users/sabinaadamska/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1607370126481/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!






In [10]:
"""
Helper function to flatten input
from (:,28,28) to (:,784)
"""

def reshape_for_nn(orig_dataset):
    reshaped_dataset = trainset
    reshaped_dataset = reshaped_dataset.reshape(*reshaped_dataset.shape[:1], -1)
    return reshaped_dataset
    
X_train = reshape_for_nn(trainset)
X_test = reshape_for_nn(testset)


In [11]:
"""
Helper function for one-hot encoding
label_range - number of categories
label_set - 1D array of target labels
""" 

def labels_one_hot(label_range,label_set):
    label_range = np.arange(10)
    label_set = np.array(y_train, ndmin=2).T
    one_hot = (label_range==label_set).astype(np.int)
    return one_hot

one_hot_y_train = labels_one_hot(10,y_train)
one_hot_y_test = labels_one_hot(10,y_test)
one_hot_y_test.shape

(60000, 10)

In [12]:
"""
Helper function to normalize the data from range (0,255) to (0,1)
Use after calling reshape_for_nn
"""
def normalize_data(data):
    norm = 1 / 255
    norm_data = np.asfarray(data[:]) * norm
    return norm_data
# Normalizing both train and test
X_train = normalize_data(X_train)
X_test = normalize_data(X_test)

In [8]:
class LinearLayer():
    def __init__(self):
        pass
    def nodes(self,nodes):
        return nodes


In [13]:
class NN2():
    def __init__(self,
                learning_rate):
        layer_nodes = LinearLayer()
        self.input_layer = layer_nodes.nodes(784)
        self.hidden_layer = layer_nodes.nodes(128)
        self.output_layer = layer_nodes.nodes(10)
        self.learning_rate =  learning_rate
        self.create_weight_matrices()
        
    def create_weight_matrices(self):
        
        self.input_hidden_matrix=np.random.rand(self.hidden_layer,self.input_layer)
        self.hidden_output_matrix=np.random.rand(self.output_layer,self.hidden_layer)
        return self.input_hidden_matrix, self.hidden_output_matrix
    
    
    def forward(self,input_vector):
        # X 
        self.input_vector=input_vector
        # X transposed to match the weights dimensions
        self.input_vector = np.array(self.input_vector, ndmin=2).T
        # dot product of input and weights 
        # hidden layer values wrapped in φ activation function
        # Transpose was achieved by changing the input order
        self.h_values = self.sigmoid(np.dot(self.input_hidden_matrix,self.input_vector))
        # ditto for output layer
        self.o_values = self.softmax(np.dot(self.hidden_output_matrix,self.h_values))
        
        return self.o_values
        
    def sigmoid(self,s):
        return 1/(1 + np.exp(-s))
    
    # https://numpy.org/doc/stable/reference/generated/numpy.exp.html
    # 1 line    
    def softmax(self,x):
        e = np.exp(x) / np.sum(np.exp(x), axis=0)
        return e
    
    # TODO: dimensions
    # Cross entrophy error/loss function
    # 1 line form the 7th tutorial
    def cross_entropy(self,target, o):
        self.entrophy = []
        for i in range(len(target)):
            e = (-target[i]*np.log(o[i])-(1.0-target[i])*np.log(1.0-o[i]))
            self.entrophy.append(e)
        return self.entrophy
    
    # https://www.youtube.com/watch?v=qB2nwJxNVxM&list=PLRqwX-V7Uu6aCibgK1PTWWu9by6XFdCfh&index=17
    # the logic follows this video 
    def update(self):
        """
        Notes on the solution
        gradients = derivative of the output = output *(1-output)
        hidden_error = np.dot(self.hidden_output_matrix,self.entrophy)
        Δ weights = lr*E*(output*(1-output))H
        E is entrophy[i] or other loss function
        H h_values or I_values
        """
        self.grad= self.o_values*(1 - self.o_values)
        self.placeholder=self.entrophy[1] * self.grad 
        self.hidden_output_matrix =self.hidden_output_matrix+self.learning_rate*np.dot(self.placeholder,self.h_values.T)
        #print("matrix",self.hidden_output_matrix)
        
        
        self.grad_h= self.h_values*(1 - self.h_values)
        self.placeholder_h=self.entrophy[0] * self.grad_h
        self.input_hidden_matrix =self.input_hidden_matrix+self.learning_rate*np.dot(self.placeholder_h,self.input_vector.T)
        #print("matrix",self.hidden_output_matrix)
             
        return self.hidden_output_matrix
    

In [14]:
# Instatiate the network
nn2= NN2(0.001) 
n = 100
# training 
count_correct = 0 
total= 0
for i in range(60):
    #i = 8
    X = X_train[i]
    y = one_hot_y_train[i]
    y_hat=nn2.forward(X)
    #loss_function = nn2.simple_error(y,y_hat)
    lossf=nn2.cross_entropy(y,y_hat)
    updat = nn2.update()

    total+=1
    if (np.argmax(y_hat) == y_train[i]):
        count_correct = count_correct + 1
        if (i%n == 0):
            break
            #print("Correct count so far: ", count_correct, "out of total so far",total)
    break
#print("Accuracy: {}".format(count_correct/total))      


#### other version of calculating the loss and updates

In [ ]:
def simple_error(self,target,o):
        #difference between prediction and labels
        self.errors = target - o
        return self.errors

    def update(self):
    # gradients = derivative of the output = output *(1-output)
    # hidden_error = np.dot(self.hidden_output_matrix,self.entrophy)
    # delta weights_h_o = lr*E *(output*(1-output))H
    # E is entrophy[i] or other loss function
    # H h_values or I_values
    # delta weights_i_h = lr*H_E *(output*(1-output))H
    
        self.grad= self.o_values*(1 - self.o_values)
        #choice of loss function
        self.placeholder=self.errors * self.grad 
        #update the weights
        print("shape pla",self.placeholder.shape)
        self.hidden_output_delta = self.learning_rate * np.dot(self.placeholder, self.h_values.T)
        #print("matrix",self.hidden_output_matrix)
        
        self.h_error = np.dot(self.hidden_output_matrix.T, output_errors * derivative_output )
        
        self.grad_h = self.h_values*(1 - self.h_values)
        self.placeholder_h = self.h_error * self.grad_h
        self.input_hidden_delta = self.learning_rate * np.dot(self.placeholder_h, self.input_vector.T)
        #print("matrix",self.hidden_output_matrix)        
        
        self.hidden_output_matrix += self.hidden_output_delta
        self.input_hidden_matrix +=self.input_hidden_delta
        
        
        return self.hidden_output_matrix